In [1]:
import pandas as pd
import numpy as np
import os
from pyspark import SparkConf #, SparkContext 
from pyspark.sql import SparkSession #, SQLContext https://spark.apache.org/docs/1.6.1/sql-programming-guide.html
from pyspark.sql import functions as F # access to the sql functions https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions
from IPython.display import HTML

In [2]:
import os
warehouse_location = os.path.abspath('../data/spark-warehouse')
java_options = "-Dderby.system.home=" + warehouse_location

conf = (SparkConf()
    .set("spark.ui.port", "4041")
    .set("spark.driver.memory", "7g")  
    .set("spark.sql.warehouse.dir", warehouse_location) # set above
    .set("hive.metastore.schema.verification", False)
    .set("javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=metastore_db;create=true")
    .set("javax.jdo.option.ConnectionDriverName", "org.apache.derby.jdbc.EmbeddedDriver")
    .set("javax.jdo.option.ConnectionUserName", 'userman')
    .set("jdo.option.ConnectionPassword", "pwd")
    .set("spark.driver.extraJavaOptions",java_options)
    .set("spark.sql.inMemoryColumnarStorage.compressed", True) # default
    .set("spark.sql.inMemoryColumnarStorage.batchSize",10000) # default
    )

spark = SparkSession.builder \
    .master("local[3]") \
    .appName('test') \
    .config(conf=conf) \
    .getOrCreate()

In [3]:
os.path.normpath("/home/jovyan/data/spark-warehouse") == warehouse_location

True

In [5]:
spark.sql('DROP DATABASE IF EXISTS irs990 CASCADE;')
spark.sql("create database weather")
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|  weather|
+---------+



In [8]:
os.getcwd()

'/home/jovyan/work'

In [9]:
spark.sql('DROP DATABASE IF EXISTS mycsv CASCADE;')
spark.sql("create database mycsv")
spark.sql("USE mycsv")
diamonds = spark.read.options(header='True'.format("csv").load("diamonds.csv")
print(diamonds.limit(5).show(5))
diamonds.write.mode("overwrite").saveAsTable("diamonds")
spark.sql('SHOW TABLES IN mycsv').show()

+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|  _c0|    _c1|  _c2|    _c3|  _c4|  _c5|  _c6| _c7| _c8| _c9|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
| 0.23|  Ideal|    E|    SI2| 61.5|   55|  326|3.95|3.98|2.43|
| 0.21|Premium|    E|    SI1| 59.8|   61|  326|3.89|3.84|2.31|
| 0.23|   Good|    E|    VS1| 56.9|   65|  327|4.05|4.07|2.31|
| 0.29|Premium|    I|    VS2| 62.4|   58|  334| 4.2|4.23|2.63|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+

None
+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|    mycsv| diamonds|      false|
+---------+---------+-----------+



In [11]:
dat = spark.table("mycsv.diamonds")
dat.toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9
0,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58,334,4.2,4.23,2.63
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57,2757,5.75,5.76,3.5
53937,0.72,Good,D,SI1,63.1,55,2757,5.69,5.75,3.61
53938,0.7,Very Good,D,SI1,62.8,60,2757,5.66,5.68,3.56
53939,0.86,Premium,H,SI2,61,58,2757,6.15,6.12,3.74
